In [84]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [85]:
import copy

In [86]:
from Types import *

In [87]:
from Functions import *

In [88]:
import postgresql

In [179]:
db = postgresql.open('pq://derin:qwerty@localhost:5432/models')

In [90]:
class GPattern:
    def __init__(self, l, textWord = "", depWordConstr = None, p = "", m = 0.0):
        self.level = l
        self.dependentWord = textWord
        if (depWordConstr == None):    
            self.dependentWordConstraints = []
        else:
            self.dependentWordConstraints = copy.copy(depWordConstr)
        self.mark = m
        self.prep = p
        self.info = ""# ????

In [91]:
#Проблема, что такое dependentWordConstrains 
#Список из условий?
#функционалов?
#Условия только, которые важны(думаю да)
#список из функций..... нет! вызов функции - долго
#просто список из элементов разных объединений

In [92]:
#GPattern содержит следующие поля:
#Level - уровень ОМУ (1, 2 или 3).
#DependentWord - зависимое слово (только для уровня 3).
#DependentWordConstraints - требования к зависимому слову (синтаксические и прочие ограничения).
#Mark - оценка модели управления
#Info – дополнительная информация о модели управления (например, где она встречалась).

In [93]:
class GPatternList:
    def __init__(self):
        self.firstLevel = []
        self.secondLevel = []
        self.thirdLevel = []

In [94]:
#GPatternList – список из GPattern, определяет список всех моделей управления данного слова или группы.

In [133]:
def extractFirstLevel(word, curMorf):
    s1 = "SELECT number_morf FROM morf_characters_of_word WHERE " + \
        "s_cl = \'" + str(curMorf.s_cl).split('.')[1] + "\' AND " + \
        "animate = \'" + str(curMorf.animate).split('.')[1] + "\' AND " + \
        "gender = \'" + str(curMorf.gender).split('.')[1] + "\' AND " + \
        "number = \'" + str(curMorf.number).split('.')[1] + "\' AND " + \
        "case1 = \'" + str(curMorf.case1).split('.')[1] + "\' AND " + \
        "reflection = \'" + str(curMorf.reflection).split('.')[1] + "\' AND " + \
        "perfective = \'" + str(curMorf.perfective).split('.')[1] + "\' AND " + \
        "transitive = \'" + str(curMorf.transitive).split('.')[1] + "\' AND " + \
        "person = \'" + str(curMorf.person).split('.')[1] + "\' AND " + \
        "tense = \'" + str(curMorf.tense).split('.')[1] + "\' AND " + \
        "voice = \'" + str(curMorf.voice).split('.')[1] + "\' AND " + \
        "degree = \'" + str(curMorf.degree).split('.')[1] + "\' AND " + \
        "static = \'" + str(curMorf.static) + "\'"
    # s1 - получение номера морфа(один морф в идеале)
    s2 = "WITH morf AS (" + s1 + "), " + \
        "num_models AS (SELECT model_1_level.number_model FROM model_1_level, morf WHERE ref_to_main_morf = morf.number_morf), " + \
        "mod AS (SELECT model_1_level.* FROM model_1_level, num_models WHERE model_1_level.number_model = num_models.number_model), " + \
        "prop AS (SELECT number_model, morf_characters_of_word.* FROM mod, morf_characters_of_word WHERE mod.ref_to_dep_morf = morf_characters_of_word.number_morf), " + \
        "imp AS (SELECT number_model, important_features.* FROM mod, important_features WHERE mod.imp_feat_dep = important_features.number_imp_feat), " + \
        "pr AS (SELECT prep_text, number_model FROM prep, mod WHERE mod.prep = prep.number_prep) " + \
        "SELECT mod.mark, pr.prep_text, imp.*, prop.* FROM imp, prop, pr, mod WHERE imp.number_model = prop.number_model AND imp.number_model = mod.number_model AND pr.number_model = mod.number_model;"
    #print(s2)
    res = db.query(s2)
    firLev = []
    shiftImpFeat = 4
    shiftProp = 19
    #print(res)
    for curConstr in res:
        curPatt = GPattern(1)
        oneModelConstr = []
        for j in range(0, 13): #13 - количество свойств в Morf
            if (curConstr[shiftImpFeat + j] == True):
                oneModelConstr.append(curConstr[shiftProp + j])
        curPatt.dependentWordConstraints.append(oneModelConstr)
        curPatt.mark = curConstr[0]
        curPatt.prep = curConstr[1]
        firLev.append(curPatt)
    #print(firLev)
    return firLev

In [134]:
def extractSecondLevel(word, curMorf):
    s0 = "SELECT number_morf FROM morf_characters_of_word WHERE " + \
        "s_cl = \'" + str(curMorf.s_cl).split('.')[1] + "\' AND " + \
        "animate = \'" + str(curMorf.animate).split('.')[1] + "\' AND " + \
        "gender = \'" + str(curMorf.gender).split('.')[1] + "\' AND " + \
        "number = \'" + str(curMorf.number).split('.')[1] + "\' AND " + \
        "case1 = \'" + str(curMorf.case1).split('.')[1] + "\' AND " + \
        "reflection = \'" + str(curMorf.reflection).split('.')[1] + "\' AND " + \
        "perfective = \'" + str(curMorf.perfective).split('.')[1] + "\' AND " + \
        "transitive = \'" + str(curMorf.transitive).split('.')[1] + "\' AND " + \
        "person = \'" + str(curMorf.person).split('.')[1] + "\' AND " + \
        "tense = \'" + str(curMorf.tense).split('.')[1] + "\' AND " + \
        "voice = \'" + str(curMorf.voice).split('.')[1] + "\' AND " + \
        "degree = \'" + str(curMorf.degree).split('.')[1] + "\' AND " + \
        "static = \'" + str(curMorf.static) + "\'"
    #получение морфа
    s1 = "WITH number_morf AS (" + s0 +"), number_word AS (SELECT number_word FROM word, number_morf  WHERE word.word_text = \'" + word + "\' AND word.ref_to_morf = number_morf.number_morf)," 
    # s1 - получение номера главного слова(одно слово в идеале)
    s2 = s1 + \
    "mod AS (SELECT * FROM model_2_level, number_word WHERE model_2_level.ref_to_main_word = number_word.number_word), prop AS (SELECT number_model, morf_characters_of_word.* FROM mod, morf_characters_of_word WHERE mod.ref_to_dep_morf = morf_characters_of_word.number_morf), imp AS (SELECT number_model, important_features.* FROM mod, important_features WHERE mod.imp_feat_dep = important_features.number_imp_feat), pr AS (SELECT prep_text, number_model FROM prep, mod WHERE mod.prep = prep.number_prep) SELECT mod.mark, pr.prep_text, imp.*, prop.* FROM imp, prop, pr, mod WHERE imp.number_model = prop.number_model AND imp.number_model = mod.number_model AND pr.number_model = mod.number_model;"

    #print(s2)
    res = db.query(s2)
    secLev = []
    shiftImpFeat = 4
    shiftProp = 19
    #print(res)
    for curConstr in res:
        curPatt = GPattern(2)
        oneModelConstr = []
        for j in range(0, 13): #13 - количество свойств в Morf
            if (curConstr[shiftImpFeat + j] == True):
                oneModelConstr.append(curConstr[shiftProp + j])
        curPatt.dependentWordConstraints.append(oneModelConstr)
        curPatt.mark = curConstr[0]
        curPatt.prep = curConstr[1]
        secLev.append(curPatt)
    #print(secLev)
    return secLev

In [135]:
def extractThirdLevel(word, curMorf):
    s0 = "SELECT number_morf FROM morf_characters_of_word WHERE " + \
        "s_cl = \'" + str(curMorf.s_cl).split('.')[1] + "\' AND " + \
        "animate = \'" + str(curMorf.animate).split('.')[1] + "\' AND " + \
        "gender = \'" + str(curMorf.gender).split('.')[1] + "\' AND " + \
        "number = \'" + str(curMorf.number).split('.')[1] + "\' AND " + \
        "case1 = \'" + str(curMorf.case1).split('.')[1] + "\' AND " + \
        "reflection = \'" + str(curMorf.reflection).split('.')[1] + "\' AND " + \
        "perfective = \'" + str(curMorf.perfective).split('.')[1] + "\' AND " + \
        "transitive = \'" + str(curMorf.transitive).split('.')[1] + "\' AND " + \
        "person = \'" + str(curMorf.person).split('.')[1] + "\' AND " + \
        "tense = \'" + str(curMorf.tense).split('.')[1] + "\' AND " + \
        "voice = \'" + str(curMorf.voice).split('.')[1] + "\' AND " + \
        "degree = \'" + str(curMorf.degree).split('.')[1] + "\' AND " + \
        "static = \'" + str(curMorf.static) + "\'"
    #получение морфа
    s1 = "WITH number_morf AS (" + s0 +"), number_word AS (SELECT number_word FROM word, number_morf  WHERE word.word_text = \'" + word + "\' AND word.ref_to_morf = number_morf.number_morf),"     
    # s1 - получение номера главного слова(одно слово в идеале)
    s2 = s1 + \
    "mod AS (SELECT * FROM model_3_level, number_word WHERE model_3_level.ref_to_main_word = number_word.number_word), w AS (SELECT number_model, word.* FROM mod, word WHERE mod.ref_to_dep_word = word.number_word), imp AS (SELECT number_model, important_features.* FROM mod, important_features WHERE mod.imp_feat_dep = important_features.number_imp_feat), pr AS (SELECT prep_text, number_model FROM prep, mod WHERE mod.prep = prep.number_prep), prop AS (SELECT * FROM morf_characters_of_word, w WHERE morf_characters_of_word.number_morf = w.ref_to_morf) SELECT mod.mark, pr.prep_text, imp.*, prop.* FROM imp, w, pr, mod, prop WHERE imp.number_model = w.number_model AND imp.number_model = mod.number_model AND pr.number_model = mod.number_model AND prop.number_model = mod.number_model;"

    #print(s2)
    res = db.query(s2)
    thLev = []
    shiftImpFeat = 4
    shiftProp = 18
    #print(res)
    for curConstr in res:
        curPatt = GPattern(3)
        oneModelConstr = []
        for j in range(0, 13): #13 - количество свойств в Morf
            if (curConstr[shiftImpFeat + j] == True):
                oneModelConstr.append(curConstr[shiftProp + j])
        curPatt.dependentWordConstraints.append(oneModelConstr)
        curPatt.mark = curConstr[0]
        curPatt.prep = curConstr[1]
        curPatt.dependentWord = curConstr[34]
        #print(curPatt.dependentWord)
        thLev.append(curPatt)
    #print(thLev)
    return thLev

In [98]:
#для модели 3 уровня нужно только зависимое слово + оценка + предлог нужны ли ограничения???(в классе нет, в базе есть)
#для модели 2 уровня - ограничения на зависимое слово + оценка + предлог
#для модели 1 уровня - ограничения на зав.слово + оценка + предлог

In [144]:
class Word:
    def morfParse(self):
        p = morph.parse(self.word)
        for curParse in p:
            self.morf.append(parseToMorf(self.word, curParse))
    def getGPatterns(self):
        curPatt = GPatternList()
        for curMorf in self.morf:
            curFirst = extractFirstLevel(self.word, curMorf)
            curSec = extractSecondLevel(self.word, curMorf)
            curThird = extractThirdLevel(self.word, curMorf)
            curPatt.firstLevel.append(curFirst)
            curPatt.secondLevel.append(curSec)
            curPatt.thirdLevel.append(curThird)
            
        self.gPatterns.append(curPatt)
    def __init__(self, name = ""):
        self.word = name #у Одинцева Word
            
        self.morf = [] # список объектов типа Morf 
        
        #список морфологических характеристик для всех вариантов морф. анализа
        self.gPatterns = []# список из GPatternList, i элемент - для i morf
        
        #с помощью морф.анализатора заполняем morf, с помощью базы - GPatterns

In [100]:
#3. Word – слово (словоформа) в исходном предложении. Word содержит следующие поля:
#Word – текстовое представление словоформы.
#Morf – список морфологических характеристик для всех вариантов морфологического анализа.
#GPatterns - метасписок моделей управления. Каждый i-тый элемент списка является списком моделей управления (структурой GPatternList) для i-го варианта морфологического анализа.

In [169]:
w = Word("ёжиться")
w.morfParse()
w.getGPatterns()

In [178]:
print(w.gPatterns[0].firstLevel[0][0].__dict__, w.gPatterns[0].firstLevel[0][1].__dict__)
print(w.gPatterns[0].secondLevel[0][0].__dict__, w.gPatterns[0].secondLevel[0][1].__dict__)
print(w.gPatterns[0].thirdLevel[0][0].__dict__, w.gPatterns[0].thirdLevel[0][1].__dict__)

print(w.morf[0].__dict__)
print(w.word)

{'mark': 1.0, 'level': 1, 'info': '', 'dependentWordConstraints': [['noun', 'instrumental']], 'dependentWord': '', 'prep': 'с'} {'mark': 1.0, 'level': 1, 'info': '', 'dependentWordConstraints': [['adverb']], 'dependentWord': '', 'prep': 'None'}
{'mark': 1.0, 'level': 2, 'info': '', 'dependentWordConstraints': [['noun', 'instrumental']], 'dependentWord': '', 'prep': 'с'} {'mark': 1.0, 'level': 2, 'info': '', 'dependentWordConstraints': [['adverb']], 'dependentWord': '', 'prep': 'None'}
{'mark': 1.0, 'level': 3, 'info': '', 'dependentWordConstraints': [['noun', 'instrumental']], 'dependentWord': 'ежом', 'prep': 'с'} {'mark': 1.0, 'level': 3, 'info': '', 'dependentWordConstraints': [['adverb']], 'dependentWord': 'быстро', 'prep': 'None'}
{'number': <Enumber.numberAny: 3>, 'person': <Eperson.personAny: 4>, 'case1': <Ecase.caseAny: 7>, 'tense': <Etense.infinitive: 1>, 'reflection': <Ereflection.reflexive: 1>, 'degree': <Edegree.degreeAny: 3>, 'static': False, 's_cl': <Es_cl.verb: 14>, 'anim

In [183]:
w = Word("с")
w.morfParse()
w.getGPatterns()

In [184]:
print(w.gPatterns[0].firstLevel[0][0].__dict__, w.gPatterns[0].firstLevel[0][1].__dict__)
print(w.gPatterns[0].secondLevel[0][0].__dict__, w.gPatterns[0].secondLevel[0][1].__dict__)
print(w.gPatterns[0].thirdLevel[0][0].__dict__, w.gPatterns[0].thirdLevel[0][1].__dict__)

print(w.morf[0].__dict__)
print(w.word)

IndexError: list index out of range

In [31]:
#s_cl animate gender number case1 reflection perfective transitive person tense voice degree static

In [223]:
class Gp:
    def __init__(self):
        self.numberModel = -1
        self.word = Word()
        self.mark = 0
        self.level = 0
    def create(self, numMod, depWord, mark1, l):
        self.numberModel = numMod
        self.word = copy.copy(depWord)
        self.mark = mark1
        self.level = l

In [103]:
class ParsePointWord:
    def __init__(self):
        self.word = Word()
        self.parsed = False
        self.usedMorfAnswer = Morf()
        self.usedGp = []#типа Gp


In [104]:
4. ParsePointWord – состояние слова в точке разбора. Поля ParsePointWord:
Word – ссылка на соответствующий объект типа Word. 
Parsed - разобрано ли слово в данной точке разбора.
UsedMorfAnswer - использованный (если слово разобрано) вариант морфологического анализа.
UsedGP – массив, состоящий из номеров моделей управления, использованных на данный момент, 
и указателей на зависимые слова (типа Word), связанные с помощью этих моделей управления.

CheckWord (PPWord, GPattern, UsedMorfAnswer). 
Эта функция проверяет, соответствует ли слово PPWord модели управления GPattern. 
Если она соответствует, то в UsedMorfAnswer помещается 
тот номер варианта морфологического разбора, при котором достигается соответствие слова тветствует модели управления.

SyntaxError: invalid syntax (<ipython-input-104-12e3b1ac3396>, line 1)

In [215]:
class ParsePoint:
    directForIsApplicable = 1
    def __init__(self):
        self.parsePointWordList = []
        self.childParsePoint = []
    def getMark(self):
        summ = 0
        for curPointWord in self.parsePointWordList:
            for curGp in curPointWord.usedGp:
                summ += curGp.mark
        return summ
    def isApplicable (mainPPWord, gPatternToApply):
    #mainPPWord - номер главного!!!
        directForIsApplicable = -directForIsApplicable
        for i in range(mainPPWord + directForIsApplicable, len(self.parsePointWordList), directForIsApplicable):
            curDepParse = self.parsePointWordList[i].word
        return (True, None, None)
    def Apply (mainPPWord, dependingPPWord, usedMorfAnswer, gPatternToApply):
        r = 0 # переписать )))
    def getNextParsePoint():
        countParsed = 0
        parsed = []
        for i in range(len(self.parsePointWordList)):
            curPointWord = self.parsePointWordList[i]
            countParsed += curPointWord.parsed
            if (curPointWord.parsed == True):
                parsed.append(i)
        if (countParsed == 0):
            for i in range(len(self.parsePointWordList)):
                curPointWord = self.parsePointWordList[i]
                for curMorf in curPointWord.word.morf:
                    if curMorf.s_cl == Es_cl.verb:
                        newParsePoint = copy.copy(self)
                        newParsePoint.parsePointWordList[i].parsed = True
                        newParsePoint.parsePointWordList[i].usedMorfAnswer = copy.copy(curMorf)
                        self.childParsePoint.append(newParsePoint)
                        return newParsePoint

            #в предложении нет глагола
            for i in range(len(self.parsePointWordList)):
                curPointWord = self.parsePointWordList[i]
                for curMorf in curPointWord.word.morf:
                    if curMorf.s_cl == Es_cl.noun and curMorf.case1 == Ecase.nominative:
                        newParsePoint = copy.copy(self)
                        newParsePoint.parsePointWordList[i].parsed = True
                        newParsePoint.parsePointWordList[i].usedMorfAnswer = copy.copy(curMorf)
                        self.childParsePoint.append(newParsePoint)
                        return newParsePoint
        else:
            bestMainWord = Word()
            bestDepWord = Word()
            usedDepMorf = Morf()
            bestModel = Gp()
            bestModelMark = 0
            for i in parsed:
                curParsedPoint = self.parsePointWordList[i]
                curWord = curParsedPoint.word
                chooseVarMorf = curParsedPoint.usedGp
                for j in range(len(curWord.morf)):
                    if (curWord.morf[j] == chooseVarMorf):
                        models = curWord.gPatterns[i]
                        break
                for curModel in (models.thirdLevel + models.secondLevel + models.firstLevel):
                    (canApply, depWord, morfDepWord) = self.isApplicable(i, curModel)
                    if (curModel.mark > bestModelMark):
                        bestModelMark = curModel.mark
                        bestModel = copy.copy(curModel)
                        bestMainWord = curWord
                        bestDepWord = depWord
                        usedDepMorf = morfDepWord
                if (bestModelMark != 0):
                    self.Apply(bestMainWord, bestDepWord, usedDepMorf, bestModel)

In [ ]:
4. ParsePoint -  структура, соответствующая точке разбора – вершине в дереве неоконченных вариантов синтаксического анализа. 
ParsePoint является списком из элементов типа ParsePointWord. 
Дополнительно ParsePoint содержит поле ChildParsePoints - список дочерних точек разбора.

In [206]:
class Sentence:
    def __init__(self): 
        self.inputStr = ""
        self.wordList = []
        self.rootPP =  ParsePoint() # ????????
        self.firstUse = True
    def setString (self, inputStr1):
        self.inputStr = inputStr1
# слово в предложении - все, отделенное пробелом, точкой, ? ! ...(смотрим только на первую .)
#: ; " ' началом предложения, запятой, (   ) тире вообще не учитываем(оно отделено пробелами), 
# дефис только в словах очень-очень и тп, 
        punctuation = [' ', '.', '?', '!', ':', ';', '\'', '\"', ',', '(', ')']
        curWord = ""
        for i in self.inputStr:
            if (i in punctuation):
                if (len(curWord) != 0):
                    self.wordList.append(Word(curWord.lower()))
                    curWord = ""
            elif (i != '-' or (len(curWord) != 0)): # - и непустое слово -  это дефис
                curWord = curWord + i
        if (len(curWord) != 0):
                self.wordList.append(Word(curWord.lower()))
                curWord = ""
    def morfParse(self):
        for curWord in self.wordList:
            curWord.morfParse()
    def getGPatterns(self):
        for curWord in self.wordList:
            curWord.getGPatterns()
    def getRootParsePoint(self):
        self.rootPP =  ParsePoint()# пока так!!! а зачем вообще эта функция???
    def getBestParsePoint(self):
        bestPoint = copy.copy(self.rootPP)
        bestPointMark = self.rootPP.getMark()
        wasChange = False
        for curChild in self.rootPP.childParsePoint:
            s1 = Sentence()
            s1.rootPP = copy.copy(curChild)
            curBest = s1.GetBestParsePoint()
            curMark = curBest.getMark()
            if (curMark > bestPointMark):
                bestPointMark = curMark
                bestPoint = curBest
                wasChange = True
        if (wasChange == False):
            return None
        return bestPoint
            
    def sintParse(self):
        if (self.firstUse == True):
            self.firstUse = False
            self.GetRootParsePoint()
        bestParsePoint = self.GetBestParsePoint()
        if (bestParsePoint == None):
            print("Больше вариантов разбора нет")
            return None
        allWordsParsed = True
        
        
        for curPointWord in bestParsePoint.parsePointWordList:
            if (curPointWord.parsed == False):
                allWordsParsed = False
                break
        while (allWordsParsed == False):
            newPoint = bestParsePoint.GetNextParsePoint()
            for curPointWord in bestParsePoint.parsePointWordList:
            if (curPointWord.parsed == False):
                allWordsParsed = False
                break
        return bestParsePoint
        
        
        

In [235]:
def f():
    print(f.i)
    f.i += 1

In [236]:
f.i = 0
f()
f()


0
1


In [207]:
s = Sentence()
s.setString("\"Дети уходите гулять\" - сказал учитель(странный человек) в громкоговоритель!")

In [208]:
s.wordList[0].__dict__

{'gPatterns': [], 'morf': [], 'word': 'дети'}

In [209]:
GetRootParsePoint (). Эта функция создает корневую точку разбора.
GetBestParsePoint (). Эта функция выдает ссылку на существующую точку разбора для построения от нее дочерней точки разбора. Выдается точка разбора с максимальной в дереве оценкой. Поиск этой точки разбора начинается с RootPP.

MorfParse (). Функция запускает морфологический анализ для всех слов в исходном тексте по очереди.
GetGPatterns (). Функция загружает модели управления для слов в предложении.
SintParse(). Функция запускает синтаксический анализ. Результатом этой функции является строка с синтаксической структурой. Каждый новый вызов этой функции выдает новый вариант анализа (если они еще есть).

SyntaxError: invalid syntax (<ipython-input-209-5ced31b7abec>, line 1)

In [216]:
str1 = input()
s = Sentence()
s.setString(str1)
s.morfParse()
s.getGPatterns()

Ёжиться с ежом


In [217]:
a100 = s.getBestParsePoint()

In [219]:
a100.__dict__

{'childParsePoint': [], 'parsePointWordList': []}

In [198]:
s.wordList[2].gPatterns[0].__dict__

{'firstLevel': [[], [], []],
 'secondLevel': [[], [], []],
 'thirdLevel': [[], [], []]}

In [127]:
morph.parse("с")[0]

Parse(word='с', tag=OpencorporaTag('PREP'), normal_form='с', score=0.998363, methods_stack=((<DictionaryAnalyzer>, 'с', 375, 0),))

In [132]:
extractThirdLevel("с", parseToMorf("с", morph.parse("с")[0]))

[]
[]


[]

In [67]:
"Звезды".lower()

'звезды'

In [6]:
"a a a".split()

['a', 'a', 'a']

In [4]:
def fun(m):
    m = [1, 2]

In [5]:
a = [4, 5]
fun(a)
print(a)

[4, 5]
